In [1]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("Daniel_Agstrand_A3_Part1")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext



In [2]:
# Q A.1.1

rdd_en = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.en',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)\
.cache() # Keep this RDD in memory!

line_count_en = rdd_en.count()

In [3]:
# Q A.1.2

rdd_sv = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.sv',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)\
.cache() # Keep this RDD in memory!

line_count_sv = rdd_sv.count()

In [4]:
# Q A.1.3

if line_count_sv == line_count_en:
    print ("Equal amount of lines! Number of lines: {}".format(line_count_en))
else:
    print ("Unequal amount of lines! Number of lines in en-sv = {} and Number of lines in sv-en = {}"\
           .format(line_count_en, line_count_sv))

Equal amount of lines! Number of lines: 1862234


In [14]:
# Q A.1.4

rdd_en_partitions_count = rdd_en.getNumPartitions()
rdd_sv_partitions_count = rdd_sv.getNumPartitions()

total_partitions_count = rdd_en_partitions_count + rdd_sv_partitions_count

print("Number of partions in rdd_en: {}\nNumber of partions in rdd_sv: {}\nNumber of total partions in all rdd: {}"\
      .format(rdd_en_partitions_count, rdd_sv_partitions_count, total_partitions_count))

Number of partions in rdd_en: 2
Number of partions in rdd_sv: 3
Number of total partions in all rdd: 5


In [30]:
# Q A.2.1

def lowercase_split(line):
    line = str(line).lower()
    line = str(line).split(" ")
    return line

rdd_en_mapped = rdd_en.map(lowercase_split) 
rdd_sv_mapped = rdd_sv.map(lowercase_split) 

In [31]:
# Q A.2.1

print("10 entries in rdd_en_mapped: " + str(rdd_en_mapped.take(10)) + "\n10 entries in rdd_vs_mapped: " + str(rdd_sv_mapped.take(10)))

10 entries in rdd_en_mapped: [['(0,', "'resumption", 'of', 'the', "session')"], ['(26,', "'i", 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', "period.')"], ['(234,', '"although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.")'], ['(426,', "'you", 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', "part-session.')"], ['(532,', '"in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,',

In [34]:
# Q A.2.2

line_count_en_mapped = rdd_en_mapped.count()
line_count_sv_mapped = rdd_sv_mapped.count()

if ((line_count_en_mapped == line_count_en) & (line_count_sv_mapped == line_count_sv)):
    print("Line count still match!")
else:
    print("Line count doesnt match any more!")

Line count still match!


In [ ]:
#spark_session.stop()